# Classification de textes

La classification de textes fait partie des applications courantes du traitement automatique du langage et, pourtant, il n’existe pas encore de méthode infaillible de parvenir à un résultat qui donne entière satisfaction, pas plus que de recette applicable à toutes les tâches.

L’objectif, extrêmement simple, continue de poser des difficultés aux programmes informatiques. Il s’agit, pour résumer, d'attribuer le plus justement une étiquette à un texte. L’email reçu est-il un spam ou un message légitime ? Cette pièce a-t-elle été écrite par Molière ou par Corneille ? La dernière critique postée sur mon blog est-elle positive ou négative ? Aux exemples de classification binaire se rajoutent des cas de classification multi-classes (à quel genre rattacher ce roman ?) et de classification multi-étiquettes (identifier par exemple qu’un message contient à la fois une réclamation pour le service client et une question pour le service technique).

Il s’agira, dans ce calepin, de reconnaître parmi un corpus de tweets ceux qui traitent de catastrophes réelles. Commençons par charger les bibliothèques nécessaires :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

## Présentation des données

Les données sont issues d’une compétition ouverte sur la plateforme *Kaggle* et servant d’introduction aux tâches de prédiction : [*Natural Language Processing with Disaster Tweets*](https://www.kaggle.com/c/nlp-getting-started/)

Le jeu de données initial est constitué de deux fichiers pour l’entraînement (7503 observations) et l’évaluation (3243 observations). Nous n’utiliserons ici que le premier.

### Contenu du jeu d’entraînement

Le jeu d’entraînement est constitué de 7613 observations décrites par cinq variables :

|Variable|Type de donnée|Description|
|:-:|:-:|-|
|`id`|entier naturel|Identifiant de l’observation|
|`keyword`|caractères|Mot-clé parmi une liste contrôlée pour qualifier le tweet|
|`location`|caractères|Provenance du tweet|
|`text`|caractères|Texte du tweet|
|`target`|entier naturel|Facteur numérique à deux niveaux pour identifier les tweets qui traitent d’une catastrophe (1) et les autres (2) |

> Addison Howard, devrishi, Phil Culliton, Yufeng Guo. (2019). [*Natural Language Processing with Disaster Tweets*](https://www.kaggle.com/c/nlp-getting-started/). Kaggle.

### Charger les données

À l’aide de *Pandas*, chargeons les données dans un *data frame* en ne reprenant que les variables `id`, `text` et `target` :

In [ ]:
df = pd.read_csv('./data/disasters.csv', usecols=['text', 'target'])

Vérifions que l’importation s’est bien déroulée :

In [ ]:
df.head()

Nous pouvons maintenant définir nos caractéristiques ainsi que la variable cible :

In [ ]:
X = df.text
y = df.target

## Vectoriser le texte

Tous les algorithmes d’apprentissage reposant sur des données numériques, il convient de choisir une méthode de vectorisation du texte. Nous optons pour une représentation des tweets sous forme de matrice TF-IDF :

In [ ]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(X)

Regardons les mesures TF-IDF du premier tweet :

> &laquo; Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all &raquo;

In [ ]:
tweet_id = 0
print(tfidf[tweet_id])

Et affichons-les dans un *data frame* :

In [ ]:
# features names
features = vectorizer.get_feature_names_out()

# remove axe of length one
arr = np.squeeze(tfidf[tweet_id].toarray())

# how many non-0 tf-idf?
nb_ids = len(arr[arr != 0])

# ids of these elements
ids = np.argsort(arr)[::-1][:nb_ids]

# retrieve feature name associated to tf-idf score
tweet_tfidf = [ (features[i], arr[i]) for i in ids ]

# vizualisation
pd.DataFrame(data=tweet_tfidf, columns=['feature', 'tfidf'])

## Réduire la dimensionnalité

La matrice obtenue est de taille respectable. D’un jeu de données de dimension $(7613, 2)$, nous sommes arrivés à une matrice $(7613, 21637)$ :

In [ ]:
print(
    f"Dimensions du dataframe : {df.shape}",
    f"Dimensions de la matrice TF-IDF : {tfidf.shape}",
    sep="\n"
)

La seule technique que nous connaissons pour réduire le nombre de dimensions est l’analyse en composantes principales. Nos données ne se prêtent malheureusement pas à l’exercice : une ACP requiert des données centrées alors qu’il n’est pas possible de centrer une matrice creuse.

Nous devons opter pour une autre technique de réduction.

### L’analyse sémantique latente

L’analyse sémantique latente (ou LSA en anglais pour *Latent Semantic Analysis*) s’attache à découvrir des concepts qui relient les documents et les termes qu’ils contiennent. Elle se fonde sur une matrice des occurrences pour effectuer ensuite une pondération TF-IDF et calculer une nouvelle matrice, par une décomposition en valeurs singulières (ou SVD en anglais pour *Singular Value Decomposition*), qui en sera une approximation mais de dimension inférieure.

Dans *Scikit-Learn*, le transformateur `TruncatedSVD()` se charge de ces aspects :

In [ ]:
# singular value decomposition in 500 components
svd = TruncatedSVD(n_components=500, random_state=42)

# transform tfidf into lsa
lsa = svd.fit_transform(tfidf)

Nous avons volontairement choisi un nombre important de composants afin d’afficher un diagramme d’éboulis en se reposant sur les attributs `.explained_variance_` et `.components_` qui enregistrent respectivement les valeurs singulières et les vecteurs singuliers.

**Attention !** Contrairement à la PCA, les valeurs singulières ne sont pas triées.

In [ ]:
# keep track of indices when singular values are sorted in reverse order
idx = svd.explained_variance_.argsort()[::-1]

# sorting
svd.explained_variance_ = svd.explained_variance_[idx]
svd.components_ = svd.components_[:, idx]

# scree plot
_ = sns.lineplot(x=range(1, len(svd.components_) + 1), y=svd.explained_variance_)
plt.title("Diagramme d’éboulis")
plt.xlabel("Numéro de la composante")
plt.ylabel("Explication de la variance")
sns.despine()

Sur le graphique, nous remarquons que la performance baisse autour de 60-70 composantes. Qu’en est-il si nous souhaitons conserver un taux de 70 % de la variance ?

In [ ]:
variance = 0
nb_c = 0
for n, singular_value in enumerate(svd.explained_variance_):
    variance += singular_value / sum(svd.explained_variance_)
    nb_c += 1
    if variance > .7: break

print(f"Les {nb_c} premières composantes expliquent {variance:.2%} de la variance.")

Conservons les 241 premières composantes :

In [ ]:
lsa = lsa[:, :nb_c]

## Évaluer la similarité entre deux documents

Nous disposons à présent d’une structure de données qui décrit 7613 documents par 241 composantes issues de la vectorisation sémantique des contenus :

In [ ]:
lsa.shape

La question qui se pose à nous maintenant est de savoir, pour un document donné, lesquels lui sont le plus similaire. Pouvoir y répondre est la première marche d’une autre tâche assez courante en traitement automatique du langage, le *clustering*. Si nous ne traiterons pas du sujet ici, nous pouvons malgré tout dresser un tableau des similarités cosinus entre tous les documents :

In [ ]:
cosine = cosine_similarity(lsa, lsa)

Cette nouvelle matrice indique, pour chaque vecteur (document), le cosinus de l’angle qu’il forme avec tous les autres dans un intervalle $[-1,1]$. Pour le premier document nous avons :

In [ ]:
cosine[0]

Dans cet aperçu nous comprenons que le premier document est colinéaire à lui-même comme le cosinus de leur angle est de 1. Regardons en dehors de lui-même quel est le document dont il est le plus proche :

In [ ]:
idx = np.argmax(cosine[0][1:])
print(
    df.text[0],
    df.text[idx],
    sep="\n"
)

Les deux semblent assez éloignés et, à raison, car leur cosinus est seulement de $0.0308$ :

In [ ]:
cosine[0][idx]

Comparons maintenant avec deux autres tweets dont la similarité cosinus est de 0.7038 :

In [ ]:
tweet_id = 144
idx = np.argmax(cosine[tweet_id][1:])

print(
    f"Tweet {tweet_id} : {df.text[tweet_id]}",
    f"Tweet {idx} : {df.text[idx]}",
    f"Similarité cosinus : {cosine[tweet_id][idx]:.4f}",
    sep="\n"
)

## Effectuer des prédictions

Il est temps de programmer un algorithme prédictif afin de confronter notre analyse à la réalité. Commençons par découper le jeu de données en jeux d’entraînement et de tests :

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(lsa, y, test_size=0.2, random_state=42)

Choisissons un algorithme de régression de logistique qui se prête bien à une tâche de classification :

In [ ]:
model = LogisticRegression(random_state=42)
_ = model.fit(X_train, y_train)

Effectuons les prédictions :

In [ ]:
y_pred = model.predict(X_test)

Et affichons le taux d’exactitude de notre programme :

In [ ]:
model.score(X_test, y_test)

## Évaluer le modèle

Pour évaluer notre modèle, nous disposons d’outils plus performants que l’exactitude. Les métriques couramment utilisées sont la précision, le rappel et la moyenne harmonique entre les deux, le résultat F1.

### La matrice de confusion

Avant de calculer les différentes mesures statistiques de la performance d’un classificateur, affichons une matrice de confusion :

In [ ]:
cfm = confusion_matrix(y_test, y_pred)
display = ConfusionMatrixDisplay(confusion_matrix=cfm, display_labels=model.classes_)
_ = display.plot()

Chaque ligne correspond à une classe réelle et chaque colonne à une classe prédite avec, sur la première ligne, la classe négative et, sur la seconde, la classe positive, tel que dans le tableau suivant :

|prédites/réelles|Classe négative|Classe positive|
|-|:-:|:-:|
|Classe négative|TN (769)|FP (105)|
|Classe positive|FN (237)|TP (412)|

Cela signifie que, sur 874 faux tweets, notre programme en a repéré 769 et que sur 649 traitant de catastrophes réelles, il n’en a identifié que 412.

### Les mesures de la performance

Pour un classificateur binaire, il est commun de ressortir les trois mesures statistiques :

- la précision (exactitude des prédictions positives) ;
- le rappel (taux de classes positives correctement étiquetées) ;
- le score F1 (compromis précision/rappel).

In [ ]:
print(
    f"Précision : {precision_score(y_test, y_pred):.2%}",
    f"Rappel : {recall_score(y_test, y_pred):.2%}",
    f"F1 score : {f1_score(y_test, y_pred):.2%}",
    sep="\n"
)

## Conclusion

Atteindre au plus vite l’objectif. C'est le motif qui nous a guidés au long de ce parcours, aussi n’avons-nous envisagé à aucun moment d’améliorer l’efficacité de notre programme. Nous aurions pu appliquer une phase de normalisation du texte en résolvant les contractions de l’anglais, en supprimant les émojis, les URLs et autres caractères indésirables ; nous aurions pu tout aussi bien vérifier le niveau de langage, la justesse de l’orthographe ou la présence de doublons ; nous aurions pu augmenter nos données avec des mesures classiques comme la longueur du texte en nombre de mots ou la longueur moyenne d’un mot ; nous aurions sans doute dû créer un sac de mots préalablement à la vectorisation en matrice TF-IDF ; et il aurait aussi été judicieux d’évaluer la pertinence de nos choix à chaque étape grâce à des mesures statistiques.